# Concise Implementation of Linear Regression

We will show you how to implement
the linear regression model
concisely by using high-level APIs

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Call upon the existing API in a framework to read data

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.6942, -0.7579],
         [ 0.2126,  0.5047],
         [-0.4384,  0.1653],
         [ 0.6413,  0.0428],
         [ 0.0198, -2.2129],
         [ 0.0310,  0.0892],
         [-1.1310, -0.7331],
         [-0.5562,  1.4170],
         [ 0.6390, -0.8279],
         [ 1.0688,  0.4248]]),
 tensor([[ 8.1669],
         [ 2.9037],
         [ 2.7676],
         [ 5.3441],
         [11.7465],
         [ 3.9521],
         [ 4.4251],
         [-1.7304],
         [ 8.2876],
         [ 4.8894]])]

Use a framework's predefined layers

In [6]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Initialize the model parameters

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm

In [8]:
loss = nn.MSELoss()

Instantiate an `SGD` instance

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

The training loop itself is strikingly similar
to what we did when implementing everything from scratch

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000195
epoch 2, loss 0.000099
epoch 3, loss 0.000099


Compare the model parameters learned by training on finite data
and the actual parameters

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0003, 0.0002])
error in estimating b: tensor([0.0007])
